# Visualization

### все модули в одной ячейке

In [ ]:
import pandas as pd
import json
from string import punctuation
from pymorphy2 import MorphAnalyzer
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
morph = MorphAnalyzer()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


# task 1
## повторно обрабатываем текст и готовим тэги для датафрейма; создаем собственно сам датафрейм

In [ ]:
with open('Vulf_Volny.60265.fb2.txt', 'r', encoding='utf-8') as f:
    book = f.read()
for i in punctuation:
    book = book.replace(i, '')
tokens = (word_tokenize(book))
words_d = {}
word_forms = []
normal_forms = []
tags = []
POSs = []
cases = []
numbers = []
genders = []
anims = []
aspects = []
invols = []
moods = []
persons = []
tenses = []
trans = []
voices = []

for token in tokens:
    annotation = morph.parse(token)
    parse = annotation[0]
    tag = parse.tag
    pos = tag.POS
    tagstr = str(tag)
    tagstr = tagstr.split(',')
    word_forms.append(parse.word)
    normal_forms.append(parse.normal_form)
    tags.append(tagstr)
    POSs.append(tag.POS)
    cases.append(tag.case)
    numbers.append(tag.number)
    genders.append(tag.gender)
    anims.append(tag.animacy)
    aspects.append(tag.aspect)
    invols.append(tag.involvement)
    moods.append(tag.mood)
    persons.append(tag.person)
    tenses.append(tag.tense)
    trans.append(tag.transitivity)
    voices.append(tag.voice)
    
words_d['word_form'] = word_forms
words_d['normal_form'] = normal_forms
words_d['POS'] = POSs
words_d['case'] = cases
words_d['number'] = numbers
words_d['gender'] = genders
words_d['animacy'] = anims
words_d['aspect'] = aspects
words_d['involvement'] = invols
words_d['mood'] = moods
words_d['person'] = persons
words_d['tense'] = tenses
words_d['transitivity'] = trans
words_d['voice'] = voices
with open('tags_table.json', 'w', encoding='utf-8') as f:
    json.dump(words_d, f, ensure_ascii=False)


data_frame = pd.DataFrame(words_d)

data_frame.to_csv('woolf.csv', sep='\t', index=False)

woolf = pd.read_csv('woolf.csv', sep='\t').fillna('')
woolf.head(40)

# task 2.1
### графики  с семинара

In [ ]:
woolf['gender'].value_counts().plot.barh(color='pink')
plt.title('Gender')
plt.xlabel('gender')
plt.ylabel('number of entries')

# task 2.2

In [ ]:
df2 = woolf[
    (woolf['gender'] != '') & (woolf['gender'] != 'мж')
][
    ['POS', 'gender', 'number']
].groupby(['POS', 'gender'], as_index=False).count()
df2.columns = ['POS', 'gender', 'total']
sns.barplot(x="POS", y="total", hue='gender', data=df2)
plt.title('Gender by POS')
plt.xlabel('POS')
plt.ylabel('number of entries')




# task 2.3

In [ ]:
len_df = woolf[['normal_form', 'POS', 'gender']
].groupby(['normal_form', 'POS'], as_index=False).count()
len_df.columns = ['normal_form', 'POS', 'total']
len_df = len_df[len_df['total'] > 10]

len_df['length'] = woolf['word_form'].apply(len)
plt.figure(figsize=(15, 5))
sns.distplot(len_df['length'], bins=20, color='blue')
plt.title('Distribution of lemma length')
plt.ylabel('%')
plt.xlabel('Length of word')




# task 4
### моя визуализация с частотностью падежей (барчарт в данном случае будет более репрезентативней)

In [ ]:
woolf['case'].value_counts().plot.bar(color='lightgreen')
plt.title('Case frequency')
plt.xlabel('cases')
plt.ylabel('number of entries')


# task 5

In [ ]:
nltk_normal = nltk.Text(list(woolf['normal_form']))
names = ["бернард", "сьюзен", "рода", "невил", "джинни", "луи"]
nltk_normal.dispersion_plot(names)